#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import users table:

In [2]:
users=pd.read_csv('C:\\Users\\User\\Desktop\\IronHack\\LABS\\lab-data-cleaning\\your-code\\users_table.csv')
users.head(2)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN


#### 3. Rename Id column to userId

In [3]:
users.rename(columns={'Id': 'userId'}, inplace = True)
users.head(1)

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN


#### 4. Import posts table:

In [4]:
posts=pd.read_csv('C:\\Users\\User\\Desktop\\IronHack\\LABS\\lab-data-cleaning\\your-code\\posts_table.csv')
posts.head(2)

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [5]:
posts.rename(columns={'Id': 'postId', 'OwnerUserId':'userId'}, inplace = True)
posts.head(1)

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [6]:

users = users[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
posts = posts[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [7]:
users.dtypes

userId        int64
Reputation    int64
Views         int64
UpVotes       int64
DownVotes     int64
dtype: object

In [8]:

users_posts = users.merge(posts, on='userId')
users_posts.shape

(38962, 9)

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [9]:
print(users_posts.isnull().sum())
print(posts.columns)

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64
Index(['postId', 'Score', 'userId', 'ViewCount', 'CommentCount'], dtype='object')


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [10]:
#i will fill the nan spaces with zeros because theres a few columns with information
users_posts['ViewCount']


users_posts['ViewCount'] = users_posts['ViewCount'].fillna(0)

print(users_posts.isnull().sum())

userId          0
Reputation      0
Views           0
UpVotes         0
DownVotes       0
postId          0
Score           0
ViewCount       0
CommentCount    0
dtype: int64


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [13]:
print(users_posts.dtypes)

users_posts['ViewCount'] = users_posts['ViewCount'].astype('int64')

print(users_posts.dtypes)

userId            int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
postId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object
userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object
